### Fractional Factorial

Improving upon Full Factorial

#### Toy Example

4 factors (A, B, C, D = AxBxC)

In [1]:
%matplotlib widget
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Settings
np.set_printoptions(precision=3)

In [ ]:
""" Generate Fractional Factorial Design """
# Define number of factors (excluding D)
n_factors = 3
levels = np.array([-1, 1])

# Generate all 2^3 combinations of A, B, C
n_runs = 2 ** n_factors
ABC = 2 * np.array([list(np.binary_repr(i, width=n_factors)) for i in range(n_runs)], dtype=int) - 1
# Create DataFrame
design = pd.DataFrame(ABC, columns=['A', 'B', 'C'])
# Define D as the product A*B*C
design['D'] = design['A'] * design['B'] * design['C']
design.insert(0, 'Run', np.arange(1, n_runs + 1))

print("Fractional Factorial Design (2^(4-1), D = AxBxC)")
display(design)

In [ ]:
""" Simulate Toy Model """
np.random.seed(42)
noise = np.random.normal(0, 2, n_runs)

design['Y'] = (50
               + 5 * design['A']
               + 3 * design['B']
               + 2 * design['C']
               + 4 * design['A'] * design['B']
               + noise)

display(design)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

""" Fit model """
model = ols('Y ~ A + B + C + D', data=design).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print("ANOVA Table")
display(anova_table)

In [ ]:
from statsmodels.graphics.factorplots import interaction_plot

""" Plot main effects and interaction """
# Main effects
fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 2, 1)
ax.set_title("Main Effects on Y")
for col in ['A', 'B', 'C', 'D']:
    means = design.groupby(col)['Y'].mean()
    ax.plot(means.index, means.values, '-o', label=col)
ax.set_xlabel('Factor Level')
ax.set_ylabel('Mean Response')
ax.set_xticks([-1,0,1])
ax.legend()
# Interaction
ax = fig.add_subplot(1, 2, 2)
interaction_plot(design['A'], design['B'], design['Y'], colors=['red', 'blue'], ax=ax)
ax.set_title('Interaction: AxB')
ax.set_xticks([-1,0,1])
fig.tight_layout()

In [ ]:
""" Coefficients and Residuals """
print("Regression Coefficients:")
display(model.params)
print("Residual Summary:")
display(model.resid.describe())

fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot()
ax.scatter(model.fittedvalues, model.resid)
ax.axhline(0, color='black', linewidth=1)
ax.set_xlabel("Fitted values")
ax.set_ylabel("Residuals")
ax.set_title("Residual Plot")